# News Article Summarization. 
This notebook prepares news articles to be inserted into the ChromaDB vector database. First, it creates a summary of the news article, then it identifies important Named Entities such as the names of politicians, locations, and relevant dates. Having a news summary allows us to quickly perform small-to-big retrieval; finding the full article from it's brief overview. This method helps us evaluate the Language Learning Model (LLM) more effectively. Additionally, the identified entities will be used as metadata and embedded with the news articles, aiding in fine-tuning and evaluating the LLM.

For more insight into this approach, check out this YouTube video by Jerry Liu, Founder of LlmamaIndex: https://youtu.be/TRjq7t2Ms5I.

## Install Libraries
Check if Huggingface transformers and required libraries are installed

In [1]:
!pip install -q transformers sentencepiece sentence-transformers datasets

## News Summary Pipeline

In [5]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

In [6]:
load_dotenv(find_dotenv())

True

In [13]:
MONGO_CONN_STRING = 'mongodb://admin:7QdQ3v0M50<>@192.168.8.166:27017/'  # os.getenv("MONGO_CONNECTION_STRING")

In [14]:
mongo_client = MongoClient(MONGO_CONN_STRING)
db = mongo_client.get_database(os.getenv("MONGO_DB_NAME"))

### Prepare Dataset

In [15]:
batch_date = {'$gte': '2024-04-05', '$lte': '2024-04-06'}

In [16]:
news_articles = list(db.get_collection('raw-news').find({'created_at': batch_date}))

In [17]:
news_articles

[{'_id': ObjectId('66106d1c9f152b08d8729d64'),
  'title': 'Politics | CNN Politics',
  'raw_content': ['Show\xa0all',
   'Show\xa0all',
   'Show\xa0all',
   '© 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. ',
   ' CNN Sans ™ & © 2016 Cable News Network.'],
  'url': 'https://edition.cnn.com/politics/',
  'source': 'CNN',
  'created_at': '2024-04-05T21:29:00.721689'},
 {'_id': ObjectId('66106d619f152b08d8729d65'),
  'title': 'House GOP pushes ahead with TikTok bill despite Trump opposition | CNN Politics',
  'raw_content': ['\n            House Republicans are pressing ahead with a Wednesday floor vote on ',
   ' unless it part ways with its Chinese parent company, even as former President ',
   'p has signaled some uneasiness over the legislation.\n    ',
   '\n            House Energy and Commerce ',
   '\xa0expressed confidence the measure would pass and downplayed the presumptive Republican presidential nominee’s opposition, noting he previously supp